Import the necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Get HTML document behind the URL of the Wikipedia page

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",'html.parser')

Parse and process the html document, extracting the table and the td tags, define list of postcodes, boroughs, neighborhoods

In [3]:
soup = BeautifulSoup(r.text[:], 'html.parser')

In [4]:
tag = soup.table

In [5]:
tables = soup.table

In [6]:
tds = tables.find_all('td')

In [7]:
postcode = tds[0::3]
postcode = list(map(lambda i:str(i)[4:7],postcode))


In [8]:
borough = tds[1::3]
borough = list(map(lambda i:str(i).split('>')[2].split('<')[0], borough))


In [9]:
neighborhood = tds[2::3]
neighborhood = list(map(lambda i:str(i).split('>')[2].split('<')[0], neighborhood))


Process the lists into a Dataframe, organize column order, delete rows with N/A borough, copy borough name for rows with N/A neighborhood

In [10]:

df = pd.DataFrame({'PostalCode':postcode, 'Borough':borough, 'Neighborhood':neighborhood} )
df = df.drop(df[df['Borough']==''].index)
mask = df['Neighborhood'] == ''
df.loc[mask, 'Neighborhood'] = df.loc[mask,'Borough']
df.reset_index(drop = True, inplace = True)
cols = df.columns.tolist()
cols = [cols[2], cols[0], cols[1]]
df = df[cols]
#Combine neighborhoods with same PostalCodes
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

Get the shape of the dataframe

In [17]:
df.shape

(100, 3)

https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value